# Building a Spam Filter with Naive Bayes
The goal of this project is to build a spam filter for SMS messages using the multinomial Naive Bayes algorithm. We aim to achieve an accuracy rate greater than 80%, meaning more than 80% of new messges will be classified correctly as either spam or non-spam.

To build our spam filter, we must:
1) Teach an algorithm how humans classify messages as either spam or non-spam.
2) Have the algorithm use that knowledge to estimate the probabilitity a new messages is either spam or non-spam.
3) Use these probabilities to classify the new message based on the following criteria: 
     - If the probability of spam is greater than non-spam, then spam
     - If the probability of non-spam is greater than spam, then non-spam
     - If the two probability are equal, then we may need a human to classify the message.

To teach the algorithm for step 1, we'll be using a dataset of 5,572 SMS messages that are already classified by humans. The dataset was put together by Tiago A. Almeida and José María Gómez Hidalgo, and it can be downloaded from the [The UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection). The data collection process is described in more details on [this page](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/#composition), where you can also find some of the papers authored by Tiago A. Almeida and José María Gómez Hidalgo.

## Exploring the Dataset
Note: The dataset uses the word *ham* to represent non-spam messages.

In [1]:
import pandas as pd
import re

sms = pd.read_csv('SMSSpamCollection', sep='\t', header=None, names=['Label', 'SMS'])

print(f'Number of SMS messages: {sms.shape[0]:,}')
print(f'Number of missing values in the dataframe: {sms.isnull().sum().sum()}\n')

def pretty_print_table(df, df_name):
    print(f'{df_name} dataset - Spam vs. ham, %')
    spam_ham_pct = round(df['Label'].value_counts(normalize=True)*100, 0)
    print(spam_ham_pct.to_markdown(tablefmt='pretty', headers=['Label', '%']))

pretty_print_table(df=sms, df_name='SMS')

sms.head()

Number of SMS messages: 5,572
Number of missing values in the dataframe: 0

SMS dataset - Spam vs. ham, %
+-------+------+
| Label |  %   |
+-------+------+
|  ham  | 87.0 |
| spam  | 13.0 |
+-------+------+


,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


## Training and Test Set
To ensure our algorithm reaches the desired accuracy rate of greater than 80%, we'll need to have a set of data to test against. Luckily, our dataset is large enough to comfortabily split into 2 parts:
- a training set used for teaching the algorithm how to classify messages.
- a test set used for checking the accuracy rate of the algorith.

We want our training set to be as large as possible while not compromising too much test data, so we'll use 80% (i.e. 4,458 messages) of the data for training, and 20% (i.e. 1,114 messages) for testing purposes.

We'll begin by randomizing the dataset to ensure that spam and ham messages are spread properly throughout both sets of data.

In [2]:
sms_randomized = sms.sample(frac=1, random_state=1)

# Creating training set (80%)
training_set = sms_randomized[:4458].reset_index(drop=True)
# Creating test set (20%)
test_set = sms_randomized[4458:].reset_index(drop=True)

We'll next analyze the percentage of spam and ham messages in the training and test datasets to ensure they're represenative of the whole dataset. We expect the percentages to be close to 87% of messages as ham, and the remaining 13% as spam.

In [3]:
pretty_print_table(df=training_set, df_name='Training')
print('\n')
pretty_print_table(df=test_set, df_name='Test')

Training dataset - Spam vs. ham, %
+-------+------+
| Label |  %   |
+-------+------+
|  ham  | 87.0 |
| spam  | 13.0 |
+-------+------+


Test dataset - Spam vs. ham, %
+-------+------+
| Label |  %   |
+-------+------+
|  ham  | 87.0 |
| spam  | 13.0 |
+-------+------+


The results look good! We can now move onto cleaning both datasets.

## Data Cleaning
To allow our algorithm to calculate the probabilities, we'll first perform some data cleaning to format the data such that we can easily extract the information we need. This format will be our current table altered in the following ways:
- All punctuation in the `SMS` column will be removed.
- All words in the `SMS` column will be converted to lowercase.
- We will add a series of new columns to represent each unique word in the vocabulary. For each row, these columns will contain the number of times their corresponding word appeared in the `SMS` column.

![New format example](images/new_format_example.png)

### Letter Case and Punctuation
First, we'll remove the punctuation and bring all the words to lower case:

In [4]:
# Before cleaning
training_set.head()

,Label,SMS
0,ham,"Yep, by the pretty sculpture"
1,ham,"Yes, princess. Are you going to make me moan?"
2,ham,Welp apparently he retired
3,ham,Havent.
4,ham,I forgot 2 ask ü all smth.. There's a card on ...


In [5]:
# After cleaning
training_set['SMS'] = training_set['SMS'].str.replace('\W', ' ', regex=True)
training_set['SMS'] = training_set['SMS'].str.lower()
training_set.head()

,Label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,havent
4,ham,i forgot 2 ask ü all smth there s a card on ...


### Creating the Vocabulary
Next, we'll create the vocabulary, as in a list of all the unique words that occur in the messages of our training set.

In [6]:
training_set['SMS'] = training_set['SMS'].str.split()

vocabulary = []
for sms in training_set['SMS']:
    for word in sms:
        vocabulary.append(word)
        
vocabulary = list(set(vocabulary))

print(f'Number of unique words in the vocabulary of the training set: {len(vocabulary):,}')

Number of unique words in the vocabulary of the training set: 7,783


### The Final Training Set
We're now going to use the vocabulary we just created to add new columns for each unique word.

In [7]:
word_counts_per_sms = {unique_word: [0] * len(training_set['SMS']) for unique_word in vocabulary}
for index, sms in enumerate(training_set['SMS']):
    for word in sms:
        word_counts_per_sms[word][index]+=1
        
word_counts = pd.DataFrame(word_counts_per_sms)
word_counts.head(3)

,damn,yay,destiny,buzz,sugardad,mm,0844,02085076972,barrel,mundhe,...,barcelona,applausestore,him,ain,kiss,nvm,praying,08452810075over18,firmware,loss
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
training_set_clean = pd.concat([training_set, word_counts], axis=1)
training_set_clean.head()

,Label,SMS,damn,yay,destiny,buzz,sugardad,mm,0844,02085076972,...,barcelona,applausestore,him,ain,kiss,nvm,praying,08452810075over18,firmware,loss
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Calculating Constants First
Now that we're done cleaning the training set, and we can begin creating the spam filter. We will do so by using the Naive Bayes algorithm to be able to classify new messages, which will need to answer these two probability questions:

![Naive Bayes Formula 1](images/Naive_Bayes_1.png)

To calculate P(wi|Spam) and P(wi|Ham) inside the formulas above, we'll need to use these equations:

![Naive Bayes Formula 2](images/Naive_Bayes_2.png)

where:
- *N<sub>w<sub>i</sub>|Spam</sub>* — the number of times the word *w<sub>i</sub>* occurs in spam messages
- *N<sub>w<sub>i</sub>|Ham</sub>* — the number of times the word *w<sub>i</sub>* occurs in ham messages
- *N<sub>Spam</sub>* — total number of words in spam messages
- *N<sub>Ham</sub>* — total number of words in ham messages
- *N<sub>Vocabulary</sub>* — total number of unique words in the vocabulary
- *α* — a smoothing parameter

Some of the terms in the four equations above will have the same value for every new message. We can calculate the value of these terms once and avoid doing the computations again when a new messages comes in. Below, we'll use our training set to calculate:
- *P(Spam)*
- *P(Ham)*
- *N<sub>Spam</sub>*
- *N<sub>Ham</sub>*
- *N<sub>Vocabulary</sub>*

We'll also use Laplace smoothing and set *α*=1.

In [9]:
# Isolating spam and ham messages first
spam_sms = training_set_clean[training_set_clean['Label'] == 'spam']
ham_sms = training_set_clean[training_set_clean['Label'] == 'ham']

# P(Spam) and P(Ham)
p_spam = len(spam_sms) / len(training_set_clean)
p_ham = len(ham_sms) / len(training_set_clean)

# N_Spam
n_words_per_spam_sms = spam_sms['SMS'].apply(len)
n_spam = n_words_per_spam_sms.sum()

# N_Ham
n_words_per_ham_sms = ham_sms['SMS'].apply(len)
n_ham = n_words_per_ham_sms.sum()

# N_Vocabulary
n_vocabulary = len(vocabulary)

# Laplace smoothing
alpha = 1

print(f'p_spam: {p_spam:.2f}\n'
      f'p_ham: {p_ham:.2f}\n'
      f'n_spam: {n_spam:,}\n'
      f'n_ham: {n_ham:,}\n'
      f'n_vocabulary: {n_vocabulary:,}\n'
      f'alpha: {alpha}')

p_spam: 0.13
p_ham: 0.87
n_spam: 15,190
n_ham: 57,237
n_vocabulary: 7,783
alpha: 1


## Calculating Parameters
The parameters *P(w<sub>i</sub>|Spam)* and *P(w<sub>i</sub>|Ham)* can vary between individual words, but they remain constant for each new message. For example, the word "free" is more likely to appear in a spam message than the word "apply", but the probability that the word "free" appears in a spam message does not change. This means we can save a lot of time by using our training set to calculate *P(w<sub>i</sub>|Spam)* and *P(w<sub>i</sub>|Ham)* for each word in our vocabulary beforehand to cut down on repeated calculations. This makes the Naive Bayes algorithm very fast compared to other algorithms, allowing us to almost instantly classify the new message.

There are 7,783 words in our vocabulary, therefore calculating both types of probabilities means we need to calculate a total of 15,566 probabilities.

The parameters are calculated using the formulas:

![Naive Bayes Formula 2](images/Naive_Bayes_2.png)

In [10]:
p_wi_spam = {}
p_wi_ham = {}

for word in vocabulary:
    p_wi_spam[word] = (spam_sms[word].sum()+alpha)/(n_spam+alpha*n_vocabulary)
    p_wi_ham[word] = (ham_sms[word].sum()+alpha)/(n_ham+alpha*n_vocabulary)

## Classifying A New Message
Now that we have all our parameters calculated, we can start creating the spam filter. The spam filter can be understood as a function that:
- Takes in as input a new message.
- Calculates *P(Spam|message)* and *P(Ham|message)* using the following formulas:
![Naive Bayes Formula 1](images/Naive_Bayes_1.png)
- Compares both values and:
    - if *P(Ham|message)* > *P(Spam|message)*, then the message is classified as ham,
    - if *P(Ham|message)* < *P(Spam|message)*, then the message is classified as spam,
    - if *P(Ham|message)* = *P(Spam|message)*, then the algorithm may request human help.
    
If a new message contains some words that are not in the vocabulary, these words will be simply ignored for calculating the probabilities.

Let's create our function and test them.

In [11]:
def classify(message, print_prob=False):
    '''Takes in a message as a string, calculates P(Spam|message) 
    and P(Ham|message), then compares the two values and classifies
    the message as spam or ham, or requires human classification. 
    '''
    
    message = re.sub('\W', ' ', message)
    message = message.lower().split()
    
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in p_wi_spam:
            p_spam_given_message *= p_wi_spam[word]
            
        if word in p_wi_ham:
            p_ham_given_message *= p_wi_ham[word]
            
    if print_prob:
        print('P(Spam|message):', p_spam_given_message)
        print('P(Ham|message):', p_ham_given_message)
    
    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_spam_given_message > p_ham_given_message:
        return 'spam'
    else:
        return 'needs human classification'

To test our function, we'll write spam and ham messages that share several words. Let's see if our algorithm can tell the difference.

In [12]:
# Spam message
classify("Urgent! Dave, you have won a free gift. Call now!", True)

P(Spam|message): 1.9489855841968855e-26
P(Ham|message): 5.055981709510022e-31


'spam'

In [13]:
# Ham message
classify("Hi Dave, can you give me a call now? It's urgent.", True)

P(Spam|message): 1.5471085373049283e-34
P(Ham|message): 2.471877453192144e-32


'ham'

Great! Looks like our algorithm classified these messages correctly.

##  Measuring the Spam Filter's Accuracy
Let's check how well the spam filter does on our test set of 1,114 messages. For the algorithm, each message in this dataset is new since we didn't use it for training. The output will be a classification label for every message, which we'll be able to compare with the actual label given by a human. 

In [14]:
test_set['Predicted'] = test_set['SMS'].apply(classify)
test_set.head()

,Label,SMS,Predicted
0,ham,Later i guess. I needa do mcat study too.,ham
1,ham,But i haf enuff space got like 4 mb...,ham
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham


Now, we'll measure the accuracy of our spam filter by comparing the predicted values with the actual ones.

In [15]:
correct = 0
total = len(test_set)
for row in test_set.iterrows():
    row = row[1]
    if row['Label'] == row['Predicted']:
        correct += 1

print('Correct:', correct)
print('Incorrect:', total - correct)
print('Accuracy:', correct/total)

Correct: 1100
Incorrect: 14
Accuracy: 0.9874326750448833


Excellent! Our spam filter looked at 1,114 messages and classified 1,100 correctly. That's an accuracy rate of 0.9874%.

## Conclusion
In this project, we managed to build a spam filter for SMS messages using the multinomial Naive Bayes algorithm. The filter has an accuracy rate of 98.74%, which exceeds our intial goal of over 80%.